In [ ]:
import gdown
import zipfile
import os
import json
import shutil
import cv2

In [ ]:
IMAGE_PATH = '/content/Dataset/Images'
LABEL_PATH = '/content/Dataset/Labels'


if not os.path.exists(IMAGE_PATH):
    os.makedirs(IMAGE_PATH)

if not os.path.exists(LABEL_PATH):
    os.makedirs(LABEL_PATH)

In [ ]:
output = gdown.download('https://drive.google.com/uc?id=1c1xi5pvwEh1vGLvainJfXrzJn-EjfenX', None, quiet=True)
print(output)

# Распаковываем архив в текущую директорию
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(IMAGE_PATH)

price_tags_images.zip


In [ ]:
# load json and save directory for labels train/val/test
json_file = '/content/via_project_13Apr2024_16h38m_json.json'
save_folder = LABEL_PATH

#source of all the images and destination folder for train/test/val
source_path = IMAGE_PATH
destination_path = os.path.join(IMAGE_PATH,'val')


# Use os.listdir() to get a list of filenames in the folder
file_names = os.listdir(source_path)

with open(json_file ) as f:
   data = json.load(f)

In [ ]:
for value in data.values():
  filename = value['filename']
  img = cv2.imread(os.path.join(source_path,filename))

  # fetching the dimensions
  wid = img.shape[1]
  hgt = img.shape[0]

  with open(os.path.join(LABEL_PATH,os.path.splitext(filename)[0]+".txt"), 'w') as annotation:
    line = ''
    for region in value['regions']:
      w = region['shape_attributes']['width']/wid
      h = region['shape_attributes']['height']/hgt
      x = region['shape_attributes']['x']/wid + w/2
      y = region['shape_attributes']['y']/hgt + h/2
      line +='0 '+ str(x) + ' ' + str(y) + ' '+ str(w) + ' '+ str(h) + '\n'
    annotation.write(line)

# Подготовка данных для обучения YOLO8n

In [ ]:
HOME = '/content/'

# Где хранятся картинки
images_dir = IMAGE_PATH

# Где хранятся метки
bbox_dir = LABEL_PATH

# Папки для датасета
dataset_path = os.path.join(HOME,'DataSetForYolo8')
train_box = os.path.join(dataset_path, 'train')
val_box = os.path.join(dataset_path,'valid')

os.makedirs(val_box, exist_ok=True)
os.makedirs(os.path.join(val_box,'images'), exist_ok=True)
os.makedirs(os.path.join(val_box,'labels'), exist_ok=True)
os.makedirs(train_box, exist_ok=True)
os.makedirs(os.path.join(train_box,'images'), exist_ok=True)
os.makedirs(os.path.join(train_box,'labels'), exist_ok=True)

In [ ]:
test_path = '/content/Dataset'
for path in os.listdir(test_path):
  full_path = os.path.join(test_path ,path)
  if os.path.isdir(full_path):
    print('Файлов в', path, 'всего', len(os.listdir(full_path)))

Файлов в Labels всего 228
Файлов в Images всего 231


In [ ]:
# Создаем датасет для сегментации
files_list = os.listdir(bbox_dir)
files_list.sort()

for i in range(len(files_list)):
  # Каждый 9 из 10 в val
  if i%10 == 9:
    shutil.move(os.path.join(bbox_dir,files_list[i]), os.path.join(val_box,'labels'))
    shutil.move(os.path.join(images_dir,files_list[i][:-3]+'jpg'),os.path.join(val_box,'images'))
  else:
    shutil.move(os.path.join(bbox_dir,files_list[i]), os.path.join(train_box,'labels'))
    shutil.move(os.path.join(images_dir,files_list[i][:-3]+'jpg'),os.path.join(train_box,'images'))

In [ ]:
CLASS_LIST = ['price tag']

In [ ]:
# Создание файла yaml
with open(os.path.join(dataset_path,'data.yaml'), 'w') as f:
  f.write('path: '+str(dataset_path))
  f.write('\ntrain: '+str(os.path.join(dataset_path,'train')))
  #f.write('\ntest: '+str(os.path.join(dataset_path,'test')))
  f.write('\nval: '+str(os.path.join(dataset_path,'valid')))

  f.write('\nnc: '+ str(len(CLASS_LIST)))

  f.write('\nnames: ' + str(CLASS_LIST))

with open(os.path.join(dataset_path,'data.yaml'), 'r') as f:
  print(f.read())

path: /content/DataSetForYolo8
train: /content/DataSetForYolo8/train
val: /content/DataSetForYolo8/valid
nc: 1
names: ['price tag']


In [ ]:
import pathlib

directory = pathlib.Path('/content/DataSetForYolo8')

with zipfile.ZipFile("DataSet_for_PriceTags.zip", mode="w") as archive:
  for file_path in directory.rglob("*"):
    archive.write(file_path,
                  arcname=file_path.relative_to(directory)
                 )